# Project: Twitter US Airline Sentiment
    
### Alfredo Coviello
    

Import the libraries, load dataset, print shape of data, data description. (5 Marks)

2. Understand of data-columns: (5 Marks)

a. Drop all other columns except “text” and “airline_sentiment”.

b. Check the shape of data.

c. Print first 5 rows of data.

3. Text pre-processing: Data preparation. (20 Marks)

a. Html tag removal.

b. Tokenization.

c. Remove the numbers.

d. Removal of Special Characters and Punctuations.

e. Conversion to lowercase.

f. Lemmatize or stemming.

g. Join the words in the list to convert back to text string in the dataframe. (So that each row contains the data in text format.)

h. Print first 5 rows of data after pre-processing.

4. Vectorization: (10 Marks)

a. Use CountVectorizer.

b. Use TfidfVectorizer.

5. Fit and evaluate model using both type of vectorization. (6+6 Marks)

6. Summarize your understanding of the application of Various Pre-processing and Vectorization and performance of your model on this dataset. (8 Marks)

## Load data

In [1]:
# Import necessary libraries.
import re, string, unicodedata
import pandas as pd
import nltk           
                        # Natural language processing tool-kit
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

!pip install contractions
import contractions


from bs4 import BeautifulSoup                 # Beautiful soup is a parsing library that can use different parsers.
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords, wordnet    # Stopwords, and wordnet corpus
from nltk.stem import LancasterStemmer, WordNetLemmatizer

[nltk_data] Downloading package punkt to /Users/acovi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/acovi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/acovi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
pd.set_option('display.max_colwidth', None)
tweets = pd.read_csv("Tweets.csv")

In [3]:
tweets

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials to the experience... tacky.,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I need to take another trip!,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing about it,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different flight to Chicago.,NaN,2015-02-22 12:01:01 -0800,NaN,NaN
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flight. No warnings or communication until we were 15 minutes Late Flight. That's called shitty customer svc,NaN,2015-02-22 11:59:46 -0800,Texas,NaN
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to #BlackBerry10,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my flight, and don't answer your phones! Any other suggestions so I can make my commitment??",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada)


In [4]:
tweets.shape

(14640, 15)

In [5]:
tweets.describe()

,tweet_id,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,1.464000e+04,14640.000000,10522.000000,14640.000000
mean,5.692184e+17,0.900169,0.638298,0.082650
std,7.791112e+14,0.162830,0.330440,0.745778
min,5.675883e+17,0.335000,0.000000,0.000000
25%,5.685592e+17,0.692300,0.360600,0.000000
50%,5.694779e+17,1.000000,0.670600,0.000000
75%,5.698905e+17,1.000000,1.000000,0.000000
max,5.703106e+17,1.000000,1.000000,44.000000


In [6]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [7]:
tweets = tweets[['text', 'airline_sentiment']]

In [8]:
tweets.head(5)

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials to the experience... tacky.,positive
2,@VirginAmerica I didn't today... Must mean I need to take another trip!,neutral
3,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",negative
4,@VirginAmerica and it's a really big bad thing about it,negative


In [9]:
tweets.shape

(14640, 2)

In [44]:
tweets.airline_sentiment.value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64


# Pre-processing

a. Html tag removal.

b. Tokenization.

c. Remove the numbers.

d. Removal of Special Characters and Punctuations.

e. Conversion to lowercase.

f. Lemmatize or stemming.

g. Join the words in the list to convert back to text string in the dataframe. (So that each row contains the data in text format.)

h. Print first 5 rows of data after pre-processing.


In [11]:
# HTML tag removal

from bs4 import BeautifulSoup

def remove_html(text):
    soup = BeautifulSoup(text, "html.parser")
    pure_text = soup.get_text()
    return pure_text

tweets['text'] = tweets["text"].apply(lambda x: remove_html(x))

/Users/acovi/Training/MachineLearningFoundations/venv2/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [12]:
tweets

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials to the experience... tacky.,positive
2,@VirginAmerica I didn't today... Must mean I need to take another trip!,neutral
3,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces & they have little recourse",negative
4,@VirginAmerica and it's a really big bad thing about it,negative
...,...,...
14635,@AmericanAir thank you we got on a different flight to Chicago.,positive
14636,@AmericanAir leaving over 20 minutes Late Flight. No warnings or communication until we were 15 minutes Late Flight. That's called shitty customer svc,negative
14637,@AmericanAir Please bring American Airlines to #BlackBerry10,neutral
14638,"@AmericanAir you have my money, you change my flight, and don't answer your phones! Any other suggestions so I can make my commitment??",negative


In [13]:
# Tokenization

tweets['tokens'] = tweets['text'].apply(lambda x: nltk.word_tokenize(x))
tweets

/Users/acovi/Training/MachineLearningFoundations/venv2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,text,airline_sentiment,tokens
0,@VirginAmerica What @dhepburn said.,neutral,"[@, VirginAmerica, What, @, dhepburn, said, .]"
1,@VirginAmerica plus you've added commercials to the experience... tacky.,positive,"[@, VirginAmerica, plus, you, 've, added, commercials, to, the, experience, ..., tacky, .]"
2,@VirginAmerica I didn't today... Must mean I need to take another trip!,neutral,"[@, VirginAmerica, I, did, n't, today, ..., Must, mean, I, need, to, take, another, trip, !]"
3,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces & they have little recourse",negative,"[@, VirginAmerica, it, 's, really, aggressive, to, blast, obnoxious, ``, entertainment, '', in, your, guests, ', faces, &, they, have, little, recourse]"
4,@VirginAmerica and it's a really big bad thing about it,negative,"[@, VirginAmerica, and, it, 's, a, really, big, bad, thing, about, it]"
...,...,...,...
14635,@AmericanAir thank you we got on a different flight to Chicago.,positive,"[@, AmericanAir, thank, you, we, got, on, a, different, flight, to, Chicago, .]"
14636,@AmericanAir leaving over 20 minutes Late Flight. No warnings or communication until we were 15 minutes Late Flight. That's called shitty customer svc,negative,"[@, AmericanAir, leaving, over, 20, minutes, Late, Flight, ., No, warnings, or, communication, until, we, were, 15, minutes, Late, Flight, ., That, 's, called, shitty, customer, svc]"
14637,@AmericanAir Please bring American Airlines to #BlackBerry10,neutral,"[@, AmericanAir, Please, bring, American, Airlines, to, #, BlackBerry10]"
14638,"@AmericanAir you have my money, you change my flight, and don't answer your phones! Any other suggestions so I can make my commitment??",negative,"[@, AmericanAir, you, have, my, money, ,, you, change, my, flight, ,, and, do, n't, answer, your, phones, !, Any, other, suggestions, so, I, can, make, my, commitment, ?, ?]"


In [14]:
tweets

,text,airline_sentiment,tokens
0,@VirginAmerica What @dhepburn said.,neutral,"[@, VirginAmerica, What, @, dhepburn, said, .]"
1,@VirginAmerica plus you've added commercials to the experience... tacky.,positive,"[@, VirginAmerica, plus, you, 've, added, commercials, to, the, experience, ..., tacky, .]"
2,@VirginAmerica I didn't today... Must mean I need to take another trip!,neutral,"[@, VirginAmerica, I, did, n't, today, ..., Must, mean, I, need, to, take, another, trip, !]"
3,"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces & they have little recourse",negative,"[@, VirginAmerica, it, 's, really, aggressive, to, blast, obnoxious, ``, entertainment, '', in, your, guests, ', faces, &, they, have, little, recourse]"
4,@VirginAmerica and it's a really big bad thing about it,negative,"[@, VirginAmerica, and, it, 's, a, really, big, bad, thing, about, it]"
...,...,...,...
14635,@AmericanAir thank you we got on a different flight to Chicago.,positive,"[@, AmericanAir, thank, you, we, got, on, a, different, flight, to, Chicago, .]"
14636,@AmericanAir leaving over 20 minutes Late Flight. No warnings or communication until we were 15 minutes Late Flight. That's called shitty customer svc,negative,"[@, AmericanAir, leaving, over, 20, minutes, Late, Flight, ., No, warnings, or, communication, until, we, were, 15, minutes, Late, Flight, ., That, 's, called, shitty, customer, svc]"
14637,@AmericanAir Please bring American Airlines to #BlackBerry10,neutral,"[@, AmericanAir, Please, bring, American, Airlines, to, #, BlackBerry10]"
14638,"@AmericanAir you have my money, you change my flight, and don't answer your phones! Any other suggestions so I can make my commitment??",negative,"[@, AmericanAir, you, have, my, money, ,, you, change, my, flight, ,, and, do, n't, answer, your, phones, !, Any, other, suggestions, so, I, can, make, my, commitment, ?, ?]"


In [15]:
# Remove numbers and special characters and punctation, and convert to lowercase

def remove_numbers_and_special_chars(list_text):
    pattern = r'[^a-zA-Z]'
    word_list = []
    for t in list_text:
        text = re.sub(pattern, '', t)
        word_list.append(text.lower())
    return word_list

tweets['no_numbers'] = tweets["tokens"].apply(lambda x: remove_numbers_and_special_chars(x))

/Users/acovi/Training/MachineLearningFoundations/venv2/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [16]:
tweets['no_numbers']

0                                                                                                                                           [, virginamerica, what, , dhepburn, said, ]
1                                                                                                  [, virginamerica, plus, you, ve, added, commercials, to, the, experience, , tacky, ]
2                                                                                                [, virginamerica, i, did, nt, today, , must, mean, i, need, to, take, another, trip, ]
3                                      [, virginamerica, it, s, really, aggressive, to, blast, obnoxious, , entertainment, , in, your, guests, , faces, , they, have, little, recourse]
4                                                                                                                  [, virginamerica, and, it, s, a, really, big, bad, thing, about, it]
                                                                                

In [17]:
# Lemmatize / stemming

def stem_words(words):
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

tweets['stems'] = tweets['no_numbers'].apply(lambda x: stem_words(x) )

/Users/acovi/Training/MachineLearningFoundations/venv2/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [18]:
tweets.stems

0                                                                                                                                  [, virginameric, what, , dhepburn, said, ]
1                                                                                                     [, virginameric, plu, you, ve, ad, commerc, to, the, expery, , tacky, ]
2                                                                                           [, virginameric, i, did, nt, today, , must, mean, i, nee, to, tak, anoth, trip, ]
3                                                 [, virginameric, it, s, real, aggress, to, blast, obnoxy, , entertain, , in, yo, guest, , fac, , they, hav, littl, recours]
4                                                                                                           [, virginameric, and, it, s, a, real, big, bad, thing, about, it]
                                                                                         ...                                      

In [19]:
def lemmatize(words):
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word)
        lemmas.append(lemma)
    return lemmas

tweets['lemas'] = tweets['no_numbers'].apply(lambda x: lemmatize(x))

/Users/acovi/Training/MachineLearningFoundations/venv2/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [20]:
tweets.lemas

0                                                                                                                                        [, virginamerica, what, , dhepburn, said, ]
1                                                                                                [, virginamerica, plus, you, ve, added, commercial, to, the, experience, , tacky, ]
2                                                                                             [, virginamerica, i, did, nt, today, , must, mean, i, need, to, take, another, trip, ]
3                                     [, virginamerica, it, s, really, aggressive, to, blast, obnoxious, , entertainment, , in, your, guest, , face, , they, have, little, recourse]
4                                                                                                               [, virginamerica, and, it, s, a, really, big, bad, thing, about, it]
                                                                                            ...

In [21]:
tweets['lemas_one'] = tweets['lemas'].apply(lambda x: ' '.join(x))
tweets['stems_one'] = tweets['stems'].apply(lambda x: ' '.join(x))

/Users/acovi/Training/MachineLearningFoundations/venv2/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/acovi/Training/MachineLearningFoundations/venv2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
tweets.lemas_one.head(5)


0                                                                                         virginamerica what  dhepburn said 
1                                                       virginamerica plus you ve added commercial to the experience  tacky 
2                                                       virginamerica i did nt today  must mean i need to take another trip 
3     virginamerica it s really aggressive to blast obnoxious  entertainment  in your guest  face  they have little recourse
4                                                                     virginamerica and it s a really big bad thing about it
Name: lemas_one, dtype: object

In [23]:
tweets.stems_one.head(5)

0                                                                       virginameric what  dhepburn said 
1                                                virginameric plu you ve ad commerc to the expery  tacky 
2                                         virginameric i did nt today  must mean i nee to tak anoth trip 
3     virginameric it s real aggress to blast obnoxy  entertain  in yo guest  fac  they hav littl recours
4                                                     virginameric and it s a real big bad thing about it
Name: stems_one, dtype: object

In [24]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               14640 non-null  object
 1   airline_sentiment  14640 non-null  object
 2   tokens             14640 non-null  object
 3   no_numbers         14640 non-null  object
 4   stems              14640 non-null  object
 5   lemas              14640 non-null  object
 6   lemas_one          14640 non-null  object
 7   stems_one          14640 non-null  object
dtypes: object(8)
memory usage: 915.1+ KB


In [25]:
# Vectorization

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=5000)
vec_stems_data_features = vectorizer.fit_transform(tweets['stems_one'])
vec_stems_data_features = vec_stems_data_features.toarray()

vec_lemas_data_features = vectorizer.fit_transform(tweets['lemas_one'])
vec_lemas_data_features = vec_lemas_data_features.toarray()


In [26]:
vec_stems_data_features.shape

(14640, 5000)

In [27]:
vec_lemas_data_features.shape

(14640, 5000)

In [40]:
vec_stems_data_featuress_data_features

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [28]:
# Fit and evaluate the model
# Model 1: stemming countvectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

labels = tweets['airline_sentiment']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vec_stems_data_features,labels, test_size=0.3, random_state=7)

In [29]:
X_train.shape

(10248, 5000)

In [30]:
X_test.shape

(4392, 5000)

In [31]:
y_train.shape

(10248,)

In [32]:
y_test.shape

(4392,)

In [33]:
forest = RandomForestClassifier(n_estimators=50, n_jobs=4)

forest = forest.fit(X_train, y_train)

print("stemming countvectorizer")
print(forest)

print(np.mean(cross_val_score(forest, vec_stems_data_features, labels, cv=10)))

stemming countvectorizer
RandomForestClassifier(n_estimators=50, n_jobs=4)
0.7379781420765028


In [34]:
# Model 2: lemmatizer countvectorizer

X_train, X_test, y_train, y_test = train_test_split(vec_lemas_data_features,labels, test_size=0.3, random_state=7)

forest = RandomForestClassifier(n_estimators=50, n_jobs=4)

forest = forest.fit(X_train, y_train)

print("lemmatizer countvectorizer")
print(forest)

print(np.mean(cross_val_score(forest, vec_lemas_data_features, labels, cv=10)))

lemmatizer countvectorizer
RandomForestClassifier(n_estimators=50, n_jobs=4)
0.735860655737705


In [35]:
# TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
tfidf_stems_data_features = vectorizer.fit_transform(tweets['stems_one'])
tfidf_stems_data_features = tfidf_stems_data_features.toarray()

tfidf_lemas_data_features = vectorizer.fit_transform(tweets['lemas_one'])
tfidf_lemas_data_features = tfidf_lemas_data_features.toarray()


In [36]:
tfidf_stems_data_features.shape

(14640, 5000)

In [37]:
tfidf_lemas_data_features.shape

(14640, 5000)

In [38]:
# Model 3: stemming tfidfvectorizer

X_train, X_test, y_train, y_test = train_test_split(tfidf_stems_data_features,labels, test_size=0.3, random_state=7)

forest = RandomForestClassifier(n_estimators=50, n_jobs=4)

forest = forest.fit(X_train, y_train)

print("stemming tfidfvectorizer")
print(forest)

print(np.mean(cross_val_score(forest, tfidf_stems_data_features, labels, cv=10)))

stemming tfidfvectorizer
RandomForestClassifier(n_estimators=50, n_jobs=4)
0.7295765027322404


In [39]:
# Model 4: lemmatizer tfidfvectorizer

X_train, X_test, y_train, y_test = train_test_split(tfidf_lemas_data_features,labels, test_size=0.3, random_state=7)

forest = RandomForestClassifier(n_estimators=50, n_jobs=4)

forest = forest.fit(X_train, y_train)

print("lemmatizer tfidfvectorizer")
print(forest)

print(np.mean(cross_val_score(forest, tfidf_lemas_data_features, labels, cv=10)))

lemmatizer tfidfvectorizer
RandomForestClassifier(n_estimators=50, n_jobs=4)
0.7261612021857923


## Summary

The corpus is a list of short texts, usually one or a few sentences (tweets).
The methods rely in using human language words as features of the texts. For this reason, and to minimize the number of features, a number of pre-processing techniques were applied:
- removal of HTML tags
- tokenization, or splitting of sentences into words
- removal of special character and numbers
- conversion to lowercase, so that character case does not create different words

Also, lemmatization and stemming were used for the same reason, to reduce the universe of words to common words roots and reduce the number of features.

Words need to be converted into numbers, or vectors, which become the features for the classification model.
I choose to try both approaches to see how they behaved.
Count Vectorizer, or bag of words, simply counts the number of times each word (token) appears.
TF IDF Vectorizer, combines the count of times a word appears in the text (more times, more important) and the number of documents of the corpus where it appears (less times, more important = it's less common, more unique and more important as a distinctive feature of the documents where it appears).

The 4 models had very similar performances. Vectorization type or stem vs lema "rooting" did not show a meaningful difference. This might be due to the short lenght of the documents, i.e. tweets, and the nature of the language, i.e. vocabulary is not too varied.
